In [2]:
suppressMessages(require(GenomicRanges))
suppressMessages(require(dplyr))
suppressMessages(require(stringr))
suppressMessages(require(optparse))
suppressMessages(require(data.table))

#Rscript /home/ahe/tools/RiboReadCountByFrame/RiboReadCountByFrame_V4.R -f /home/ahe/Analysis/201903_Fleur/data/ribo/191218_out/list.txt -p /home/fleur/Analysis/191210_GTFfiles/GBM/GBM_final.gtf -r /home/ahe/Analysis/201903_Fleur/data/RNA/191127/GSC_transcript.gtf -o /home/ahe/testfolder/test_hg38

In [2]:
script.name="/home/ahe/tools/RiboReadCountByFrame/RiboReadCountByFrame_V2.R"
script.basename <- dirname(script.name)

source(paste0(script.basename,"/Codes/functions.R"))

opt=list(file="/home/ahe/Analysis/201903_Fleur/data/ribo/191218_out/list.txt",
         peptide_gtf="/home/fleur/Analysis/191210_GTFfiles/GBM/GBM_final.gtf",
         rna_gtf="/home/ahe/Analysis/201903_Fleur/data/RNA/191127/GSC_transcript.gtf",
         out="/home/ahe/testfolder/test_hg38")

ptm <- proc.time()

In [8]:
script.name="/home/ahe/tools/RiboReadCountByFrame/RiboReadCountByFrame.R"
script.basename <- dirname(script.name)

source(paste0(script.basename,"/Codes/functions.R"))

opt=list(file="/home/ipatta/data_analysis/ribodata_Indu/01282020_out/All-corrected.Sam-files/FM-T-input-downSamp-sam.txt",
         peptide_gtf="/home/ipatta/data_analysis/ribodata_Indu/01282020_out/merged.ORF.gtf/repre.valid.ORF.final.gtf",
         rna_gtf="/home/ipatta/data_analysis/ribodata_Indu/01232012_totalRNA/01232020_out/FM-T-Final-transcript.gtf",
         out="/home/ahe/testfolder/test_indu")

ptm <- proc.time()

In [3]:
ORF=fread("/home/ahe/Analysis/201903_Fleur/data/ribo/191218_mouse_out/all_predicted_ORF.gtf",sep="\t",data.table=F)
ORF=ORF[ORF$V1=="chr16" & ORF$V4>25137409 & ORF$V5<25189900,]
write.table(ORF,"/home/ahe/testfolder/ORF.gtf",sep="\t",col.names = F,row.names = F,quote=F)

rna_gtf=fread("/home/ahe/Analysis/201903_Fleur/data/RNA/190408_445B/proBcell_transcript.gtf",sep="\t",data.table=F,fill=T)
rna_gtf=rna_gtf[rna_gtf$V1=="chr16" & rna_gtf$V4>25137409 & rna_gtf$V5<25189900,]
write.table(rna_gtf,"/home/ahe/testfolder/RNA.gtf",sep="\t",col.names = F,row.names = F,quote=F)

In [63]:
script.name="/home/ahe/tools/RiboReadCountByFrame/RiboReadCountByFrame_V2.R"
script.basename <- dirname(script.name)

source(paste0(script.basename,"/Codes/functions.R"))

opt=list(file="/home/ahe/testfolder/list.txt",
         peptide_gtf="/home/ahe/testfolder/ORF.gtf",
         rna_gtf="/home/ahe/testfolder/RNA.gtf",
         out="/home/ahe/testfolder/test_mm10")

ptm <- proc.time()

In [64]:
# ---------------------------
# Step 1: Create read bed file from sam file
cat("Preprorcessing and filtering sam file(s) ... ")

#setting up
sh_name = paste0(script.basename, "/Codes/sam_to_bed_strand_v2.sh")
read_basename = gsub("\\..*$","",basename(opt$file))
pre_bed_name = paste0(script.basename, "/Temp/", read_basename, "_pre_bedtools_intersect.tsv")

file_vec=fread(opt$file,data.table=F,header=F)[,1]
id_vec=gsub("\\..*$","",basename(file_vec))
cat(paste0(length(file_vec)," in total\n"))

# Create gtf bed file and filter exon for pre bedtools intersect in linux
repre_gtf_bed = paste0(script.basename, "/Temp/", read_basename, "_repre_gtf2bed.bed")
repre_gtf_bed_m = paste0(script.basename, "/Temp/", read_basename, "_repre_gtf2bed_merged.bed")

# need a lot of \ to escape quotation mark 
# uniq again, becasue orfID can have many same exon coordinate > falsely increase read count
system(paste("grep \'exon\\s\'", opt$peptide_gtf, "| awk \'{print $1 \"\t\" $4 \"\t\" $5 \"\t\" $7}\' | uniq | sort -k1,1 -k2,2n >", repre_gtf_bed))
system(paste("bedtools merge -i ",repre_gtf_bed," >", repre_gtf_bed_m))


post_bed_name=rep("",length(file_vec))
for(i in 1:length(file_vec)){
    cat(paste0("file ",i," ... "))

    system(paste(sh_name, file_vec[i], pre_bed_name))

    # Run bedtools intersect, -s: match strand, -wa: if A intersect B, keep A
    post_bed_name[i] = paste0(script.basename, "/Temp/", read_basename,"_",id_vec[i], "_post_bedtools_intersect.tsv")

    #prefilter the reads to reduce memory usage by R
    system(paste("bedtools intersect -a", pre_bed_name, "-b", repre_gtf_bed_m, "-wa>", post_bed_name[i]))
    
    t=second_to_humanReadableTime(round((proc.time() - ptm)[3],2)) 
    cat(paste0(t[1],"h-",t[2],"m-",round(t[3]),"s\n"))
}

#rm preocessed gtf file
system(paste("rm",repre_gtf_bed,pre_bed_name,repre_gtf_bed_m))


Preprorcessing and filtering sam file(s) ... 1 in total
file 1 ... 0h-0m-1s


In [65]:
# -----------------------------
# Step 2: getting information from RNA-gtf
cat("Getting info from the RNA reference gtf file ... ")

RNA_gtf = fread(opt$rna_gtf, sep="\t", stringsAsFactors=F,header=F,data.table=F,fill=T)
RNA_gtf = RNA_gtf[RNA_gtf$V3 == "exon",]
RNA_gtf = str_split_fixed(RNA_gtf$V9, "\"; ",6) #split info to cloumns

#seperate gtf by whether they have gene name
nc_idx=!grepl("gene_name",RNA_gtf[,4])
nc_gtf = RNA_gtf[nc_idx,]
RNA_gtf = RNA_gtf[!nc_idx,]
nt_idx=grepl("exon_number",RNA_gtf[,3])
nt_gtf = RNA_gtf[nt_idx,]
RNA_gtf = RNA_gtf[!nt_idx,]

#process known RNA
RNA_gtf = data.frame(RNA_gtf[,1:5],stringsAsFactors=F)
colnames(RNA_gtf)=c("gene_id","transcript_id","gene_type","gene_name","transcript_type")
RNA_gtf$gene_id=substr(RNA_gtf$gene_id,10,nchar(RNA_gtf$gene_id))
RNA_gtf$transcript_id=substr(RNA_gtf$transcript_id,16,nchar(RNA_gtf$transcript_id))
RNA_gtf$gene_type=substr(RNA_gtf$gene_type,12,nchar(RNA_gtf$gene_type))
RNA_gtf$gene_name=substr(RNA_gtf$gene_name,12,nchar(RNA_gtf$gene_name))
RNA_gtf$transcript_type=substr(RNA_gtf$transcript_type,18,nchar(RNA_gtf$transcript_type))
RNA_gtf=unique(RNA_gtf)

#process new transcript of known RNA
nt_gtf = data.frame(nt_gtf[,1:5],stringsAsFactors=F)
colnames(nt_gtf)=c("gene_id","transcript_id","gene_type","gene_name","transcript_type")
if(nrow(nt_gtf)>0){
    nt_gtf$gene_id=substr(nt_gtf$gene_id,10,nchar(nt_gtf$gene_id))
    nt_gtf$transcript_id=substr(nt_gtf$transcript_id,16,nchar(nt_gtf$transcript_id))
    nt_gtf$gene_type=RNA_gtf$gene_type[match(nt_gtf$gene_name,RNA_gtf$gene_name)]
    nt_gtf$gene_name=substr(nt_gtf$gene_name,12,nchar(nt_gtf$gene_name))
    nt_gtf$transcript_type="novo_transcript_known_gene"
    nt_gtf=unique(nt_gtf)
}

#process novo RNA
nc_gtf = data.frame(nc_gtf[,1:2],stringsAsFactors=F)
colnames(nc_gtf)=c("gene_id","transcript_id")
if(nrow(nc_gtf)>0){
    nc_gtf$gene_id=substr(nc_gtf$gene_id,10,nchar(nc_gtf$gene_id))
    nc_gtf$transcript_id=substr(nc_gtf$transcript_id,16,nchar(nc_gtf$transcript_id))
    nc_gtf=unique(nc_gtf)
    nc_gtf$gene_type="novo_ncRNA"
    nc_gtf$gene_name=paste0("novo_ncRNA",1:nrow(nc_gtf))
    nc_gtf$transcript_type="novo_ncRNA"
}

#merge known RNA & novo RNA
RNA_gtf=rbind(RNA_gtf,nt_gtf,nc_gtf)
rm(nc_gtf,nt_gtf)

t=second_to_humanReadableTime(round((proc.time() - ptm)[3],2)) 
cat(paste0(t[1],"h-",t[2],"m-",round(t[3]),"s\n"))

Getting info from the RNA reference gtf file ... 0h-0m-2s


In [66]:
# -----------------------------
# Step 3: Read in "repre.valid.ORF.*gtf" and prepare for intersect
cat("Getting info from the peptide gtf file ... ")

gtf = fread(opt$peptide_gtf, sep="\t", stringsAsFactors=F,header=F,data.table=F)

# Remove duplicated rows, if input is from multiple gtf file concatenated from multiple samples
gtf = distinct(gtf)
gtf=gtf[gtf$V3 == "CDS",]

# Select the columns we need and filter only "CDS" 
gtf_bed = gtf[,c(1, 4, 5, 8, 9, 7)]
colnames(gtf_bed) = c("chr", "str", "end", "frame", "info", "strand_col")
gtf_bed = unique(gtf_bed)

tmp=str_split_fixed(gtf_bed$info, "\"; ",2)
tmp=tmp[,1]
gtf_bed$orf_id=substr(tmp,10,nchar(tmp))
rm(tmp)

#reorder
gtf_bed=gtf_bed[,c("chr", "str", "end", "frame", "orf_id", "strand_col")]

# change to appropiate column data type
gtf_bed$frame = as.numeric(gtf_bed$frame)

gtf_bed = arrange(gtf_bed, orf_id)

t=second_to_humanReadableTime(round((proc.time() - ptm)[3],2)) 
cat(paste0(t[1],"h-",t[2],"m-",round(t[3]),"s\n"))

Getting info from the peptide gtf file ... 0h-0m-3s


In [67]:
# -------------------------
# Step 4: Create a merged ORF table for annotation and a dataframe for intersect
cat("Generating merged_ORF_id for peptide ... ")

# return list of positions for each element
index = split(seq_along(gtf_bed$orf_id), gtf_bed$orf_id)

mORF_vec = rep("",length(index))
for (i in 1:length(index)){
    
    idx_vec = index[[i]]
    
    str = sapply(as.character(gtf_bed$str[idx_vec]), paste0, ":")
    end = sapply(as.character(gtf_bed$end[idx_vec]), paste0, ":")
    frame = sapply(as.character(gtf_bed$frame[idx_vec]), paste0, "|")
    
    chr = paste0(gtf_bed$chr[idx_vec[1]], ";")
    strand = gtf_bed$strand[idx_vec[1]]
    
    mORF = paste(c(rbind(str, end, frame)), collapse="")
    mORF = paste0(mORF, chr, strand)
    
    # name is first old orf_id, value is string of CDSs
    mORF_vec[i] = mORF
    names(mORF_vec)[i]=gtf_bed$orf_id[idx_vec[1]]
    }

t=second_to_humanReadableTime(round((proc.time() - ptm)[3],2)) 
cat(paste0(t[1],"h-",t[2],"m-",round(t[3]),"s\n"))

Generating merged_ORF_id for peptide ... 0h-0m-3s


In [68]:
# ------------------------
# Step 5: Create Annotation table for merged_ORF to geneID
cat("Creating annotation table ... ")

# Create mORF_table by adding column "geneID" "tx_id" "merged_orfID" > for annotation
tx_id_vec = str_split_fixed(gtf_bed$orf_id, ":",2) #split info to cloumns
tx_id_vec = tx_id_vec[,1]

idx = match(tx_id_vec, RNA_gtf$transcript_id)

mORF_table = gtf_bed %>%
            mutate(tx_id = tx_id_vec, ens_gene = RNA_gtf$gene_id[idx], ext_gene = RNA_gtf$gene_name[idx], 
                   merged_orfID = as.character(mORF_vec[gtf_bed$orf_id]))

# Use distinct() to merge same merged_ORF (CDSs string)
mORF_table = mORF_table %>%
    select(tx_id, ens_gene, ext_gene, merged_orfID) %>%  # Note: merged_ORF can match many tx_id, here just show one
    distinct(merged_orfID, .keep_all = TRUE) %>%
    arrange(ext_gene)

# create a vector of number to add to geneID, eg. A2M.1, A2M.2, A2M.3, AAAS.1 ...
gene_nvec=occurrance_count_4_each_element(mORF_table$ext_gene)

#identify NAs
ext_na_idx=is.na(mORF_table$ext_gene)
ens_na_idx=is.na(mORF_table$ens_gene)
mORF_table$ens_gene[ens_na_idx] = "NA"
mORF_table$ext_gene[ext_na_idx] = "NA"
if(sum(ext_na_idx)>0 | sum(ens_na_idx)>0){warning("NA in matching transcript id, didn't you use the RNA-gtf to generate the peptide-gtf, rerun ribORF with current RNA-gtf")}

# Add number to geneID
mORF_table$simpID = paste0(mORF_table$ext_gene, ".", gene_nvec)

t=second_to_humanReadableTime(round((proc.time() - ptm)[3],2)) 
cat(paste0(t[1],"h-",t[2],"m-",round(t[3]),"s\n"))

Creating annotation table ... 0h-0m-4s


In [69]:
# ---------------------
# Step 6: Create a merged ORF dataframe for intersect
cat("Creating intersectable table ... ")
mORF_unique = mORF_vec[!duplicated(mORF_vec)]

CDS_blocks=gtf_bed
CDS_blocks=CDS_blocks[CDS_blocks$orf_id %in% names(mORF_unique),]
CDS_blocks$full_id=mORF_vec[match(CDS_blocks$orf_id,names(mORF_vec))]
matching_vec=match(CDS_blocks$full_id,mORF_table$merged_orfID)
CDS_blocks$simp_id=mORF_table$simpID[matching_vec]
CDS_blocks$gene_id=mORF_table$ext_gene[matching_vec]
CDS_blocks$genetype=RNA_gtf$gene_type[match(CDS_blocks$gene_id,RNA_gtf$gene_name)]
#CDS_blocks$simp_id_count=occurrance_count_4_each_element(CDS_blocks$simp_id)
#CDS_blocks$exon_id=paste0(CDS_blocks$simp_id,".",CDS_blocks$simp_id_count)

mORF_df=CDS_blocks[,c("chr","str","end","frame","simp_id","strand_col","gene_id")]
mGENE_df=mORF_df[!duplicated(mORF_df[,c("chr","str","end","gene_id")]),]
mGENE_df=mGENE_df[!mGENE_df$gene_id =="NA",]
gene_chr_tbl=unique(mGENE_df[,c("gene_id","chr")])
mGENE_df=bedtools_merge_by_gene(mGENE_df,gene_chr_tbl,gene_col=7,strand_col=6,ignore_strand=F)
colnames(mGENE_df)[4]="strand_col"

t=second_to_humanReadableTime(round((proc.time() - ptm)[3],2)) 
cat(paste0(t[1],"h-",t[2],"m-",round(t[3]),"s\n"))

Creating intersectable table ... 0h-0m-4s


In [70]:
CDS_blocks

,chr,str,end,frame,orf_id,strand_col,full_id,simp_id,gene_id,genetype
1,chr16,25146914,25147010,1,TCONS_00058454:chr16:-|6|277:100:217|noncoding|ATG,-,25146914:25147010:1|25178043:25178059:0|chr16;-,novo_ncRNA5.1,novo_ncRNA5,novo_ncRNA
2,chr16,25178043,25178059,0,TCONS_00058454:chr16:-|6|277:100:217|noncoding|ATG,-,25146914:25147010:1|25178043:25178059:0|chr16;-,novo_ncRNA5.1,novo_ncRNA5,novo_ncRNA
5,chr16,25162479,25162518,1,TCONS_00058457:chr16:-|10|4313:146:206|noncoding|ATG,-,25162479:25162518:1|25178043:25178059:0|chr16;-,novo_ncRNA2.1,novo_ncRNA2,novo_ncRNA
6,chr16,25178043,25178059,0,TCONS_00058457:chr16:-|10|4313:146:206|noncoding|ATG,-,25162479:25162518:1|25178043:25178059:0|chr16;-,novo_ncRNA2.1,novo_ncRNA2,novo_ncRNA
7,chr16,25146914,25147006,0,TCONS_00058457:chr16:-|19|4313:246:342|noncoding|ATG,-,25146914:25147006:0|chr16;-,novo_ncRNA2.2,novo_ncRNA2,novo_ncRNA


In [74]:
i=1

    # -----------------------------
    # Step 7: Read in "FC2_Index5_*offsetCorrect_post_bedtools_intersect.bed"
    # Read in reads bed file 
    cat(paste0("input ",i," Reading reads ... "))
    reads = fread(post_bed_name[i], sep="\t", stringsAsFactors=F,data.table=F,header=F)
    colnames(reads) = c("chr", "start", "end", "strand_col")
    reads$start=reads$start +3 # add back 3, altered in sam_to_bed_strand_v2.sh
    reads$end=reads$end - 3
    #system(paste0("rm ",post_bed_name[i]))
    t=second_to_humanReadableTime(round((proc.time() - ptm)[3],2)) 
    cat(paste0(t[1],"h-",t[2],"m-",round(t[3]),"s\n"))

    # -----------------------
    # Step 8: Intersect reads and mergeGENE
    # query (reads) is the first argument, subejct(ORF) the second
    cat(paste0("input ",i," Counting read numbers in gene ... "))
    intersect_out = bedtools_intersect(mGENE_df, reads, ignore_strand=FALSE, count = T, overlap=F,maxgap = 0,minoverlap = 1)
    mGENE_df$count=intersect_out$count
    gene_count_tbl=data.table(mGENE_df)
    gene_count_tbl=gene_count_tbl[,.(count=sum(count)),by=gene_id]

    t=second_to_humanReadableTime(round((proc.time() - ptm)[3],2)) 
    cat(paste0(t[1],"h-",t[2],"m-",round(t[3]),"s\n"))



input 1 Reading reads ... 0h-15m-15s
input 1 Counting read numbers in gene ... 0h-15m-15s


In [75]:
reads

chr,start,end,strand_col
chr16,25146925,25146925,-


In [72]:
    # -----------------------
    # Step 9: Intersect reads and mergeORF
    # query (reads) is the first argument, subejct(ORF) the second
    cat(paste0("input ",i," Counting read numbers in ORF ... "))

    intersect_out = bedtools_intersect(reads, CDS_blocks, ignore_strand=FALSE)

    overlap = intersect_out$overlap

    # Use dplyr package, do all steps by piping: 
        # Add read_position, cds_start, cds_frame by index; 
        # Add remainder column by calculating (read_pos - cds_start - cds frame)/3 and output remainder
        # Add tx_id column by index

    cds_start=mORF_df$str
    cds_start[mORF_df$strand_col=="-"]=mORF_df$end[mORF_df$strand_col=="-"]

    result = overlap %>%
        mutate(read_pos = reads[overlap$bed1_idx, 2], cds_start = cds_start[overlap$bed2_idx], 
               cds_frame = mORF_df[overlap$bed2_idx, 4], remainder = (abs(read_pos - cds_start) + (3-cds_frame)) %% 3, 
               orfID = mORF_df[overlap$bed2_idx, 5])
    
    # Select result that are in frame
    result_inframe = result %>%
        filter(remainder == 0)

    result_f1 = result %>% 
        filter(remainder==0) %>%
        count(orfID)

    colnames(result_f1) = c("simpID", "f1Num")

    result_f2 = result %>% 
        filter(remainder==1) %>%
        count(orfID)

    colnames(result_f2) = c("simpID", "f2Num")

    result_f3 = result %>% 
        filter(remainder==2) %>%
        count(orfID)

    colnames(result_f3) = c("simpID", "f3Num")

    # join the three tables together
    result_f1_f2_f3 = full_join(full_join(result_f1, result_f2, by='simpID'), result_f3, by='simpID')

    # Substitue NA to 0
    result_f1_f2_f3[is.na(result_f1_f2_f3)] = 0

    # Order by f1Num
    result_f1_f2_f3 = arrange(result_f1_f2_f3, desc(f1Num))

    # Add gene IDs
    idx1 = match(result_f1_f2_f3$simpID, mORF_table$simpID)

    result_f1_f2_f3 = result_f1_f2_f3 %>%
                    mutate(ens_gene = mORF_table$ens_gene[idx1], ext_gene = mORF_table$ext_gene[idx1], merged_orfID=mORF_table$merged_orfID[idx1])

    #Re-order column, easier to see f1Num
    result_f1_f2_f3 = result_f1_f2_f3[,c(2,3,4,5,6,1,7)]

    t=second_to_humanReadableTime(round((proc.time() - ptm)[3],2)) 
    cat(paste0(t[1],"h-",t[2],"m-",round(t[3]),"s\n"))

input 1 Counting read numbers in ORF ... 0h-0m-11s


In [73]:
result_f1_f2_f3

f1Num,f2Num,f3Num,ens_gene,ext_gene,simpID,merged_orfID
1,0,0,XLOC_025760,novo_ncRNA2,novo_ncRNA2.2,25146914:25147006:0|chr16;-
1,0,0,XLOC_025760,novo_ncRNA5,novo_ncRNA5.1,25146914:25147010:1|25178043:25178059:0|chr16;-


In [ ]:
orfID   chrom   strand  codon5  codon3  length  readNum f1      f2      f3      entropy MAXentropy      PME     codonNum        f1max   pred.pvalue
TCONS_00058454:chr16:-|6|277:100:217|noncoding|ATG      chr16   -       25146910        25178059        117     27      0.815   0.074   0.111   1.869   2.885   0.648   11      0.636   0.8350
TCONS_00058456:chr16:-|10|4234:146:263|noncoding|ATG    chr16   -       25146910        25178059        117     27      0.815   0.074   0.111   1.869   2.885   0.648   11      0.636   0.8350
TCONS_00058457:chr16:-|10|4313:146:206|noncoding|ATG    chr16   -       25162475        25178059        60      13      0.923   0       0.077   0.536   2.245   0.239   4       1       1.2390


In [74]:
tmp=data.table(reads[(reads$chr=="chr16" & reads$start > 25178040 & reads$end < 25178061 ),])
tmp=tmp[,.(count=.N),by=.(chr,start,end,strand_col)]

In [24]:
bed1=mGENE_df
bed2=reads
bed1 <- GRanges(seqnames = bed1[, 1], ranges = IRanges(start = bed1[, 2], end = bed1[, 3]), strand = bed1[, "strand_col"])
bed2 <- GRanges(seqnames = bed2[, 1], ranges = IRanges(start = bed2[, 2], end = bed2[, 3]), strand = bed2[, "strand_col"])
countOverlaps(bed1, bed2, ignore.strand = F, minoverlap = 1)

[1] 15 12 15  1 12

In [20]:
bedtools_intersect

function (bed1, bed2, overlap = T, count = F, maxgap = 0, minoverlap = 0, 
    ignore_strand = T) 
{
    suppressMessages(require(GenomicRanges))
    if (ignore_strand) {
        bed1 <- GRanges(seqnames = bed1[, 1], ranges = IRanges(start = bed1[, 
            2], end = bed1[, 3]))
        bed2 <- GRanges(seqnames = bed2[, 1], ranges = IRanges(start = bed2[, 
            2], end = bed2[, 3]))
    }
    else {
        bed1 <- GRanges(seqnames = bed1[, 1], ranges = IRanges(start = bed1[, 
            2], end = bed1[, 3]), strand = bed1[, "strand_col"])
        bed2 <- GRanges(seqnames = bed2[, 1], ranges = IRanges(start = bed2[, 
            2], end = bed2[, 3]), strand = bed2[, "strand_col"])
    }
    if (overlap) {
        overlap = findOverlaps(bed1, bed2, ignore.strand = ignore_strand, 
            maxgap = maxgap, minoverlap = minoverlap)
        overlap = data.frame(overlap)
        colnames(overlap) = c("bed1_idx", "bed2_idx")
    }
    if (count) {
        count = countOverlaps(bed1, bed2, ignore.strand = ignore_strand, 
            maxgap = maxgap, minoverlap = minoverlap)
    }
    return(list(overlap = overlap, count = count))
}
<bytecode: 0x9d3d258>

In [144]:
# ----------------------------
cat(paste0("Mapping reads to ORFs ... ",length(file_vec)," input sam files\n"))
for (i in 1:length(file_vec)){
    # -----------------------------
    # Step 7: Read in "FC2_Index5_*offsetCorrect_post_bedtools_intersect.bed"
    # Read in reads bed file 
    cat(paste0("input ",i," Reading reads ... "))
    reads = fread(post_bed_name[i], sep="\t", stringsAsFactors=F,data.table=F,header=F)
    colnames(reads) = c("chr", "start", "end", "strand_col")
    system(paste0("rm ",post_bed_name[i]))
    t=second_to_humanReadableTime(round((proc.time() - ptm)[3],2)) 
    cat(paste0(t[1],"h-",t[2],"m-",round(t[3]),"s\n"))

    # -----------------------
    # Step 8: Intersect reads and mergeGENE
    # query (reads) is the first argument, subejct(ORF) the second
    cat(paste0("input ",i," Counting read numbers in gene ... "))
    intersect_out = bedtools_intersect(mGENE_df, reads, ignore_strand=FALSE, count = T, overlap=F)
    mGENE_df$count=intersect_out$count
    gene_count_tbl=data.table(mGENE_df)
    gene_count_tbl=gene_count_tbl[,.(count=sum(count)),by=gene_id]

    t=second_to_humanReadableTime(round((proc.time() - ptm)[3],2)) 
    cat(paste0(t[1],"h-",t[2],"m-",round(t[3]),"s\n"))
    # -----------------------
    # Step 9: Intersect reads and mergeORF
    # query (reads) is the first argument, subejct(ORF) the second
    cat(paste0("input ",i," Counting read numbers in ORF ... "))

    intersect_out = bedtools_intersect(reads, CDS_blocks, ignore_strand=FALSE)

    overlap = intersect_out$overlap

    # Use dplyr package, do all steps by piping: 
        # Add read_position, cds_start, cds_frame by index; 
        # Add remainder column by calculating (read_pos - cds_start - cds frame)/3 and output remainder
        # Add tx_id column by index

    result = overlap %>%
        mutate(read_pos = reads[overlap$bed1_idx, 2], cds_start = mORF_df[overlap$bed2_idx, 2], 
               cds_frame = mORF_df[overlap$bed2_idx, 4], remainder = (read_pos - cds_start - cds_frame) %% 3, 
               orfID = mORF_df[overlap$bed2_idx, 5])

    # Select result that are in frame
    result_inframe = result %>%
        filter(remainder == 0)

    result_f1 = result %>% 
        filter(remainder==0) %>%
        count(orfID)

    colnames(result_f1) = c("simpID", "f1Num")

    result_f2 = result %>% 
        filter(remainder==1) %>%
        count(orfID)

    colnames(result_f2) = c("simpID", "f2Num")

    result_f3 = result %>% 
        filter(remainder==2) %>%
        count(orfID)

    colnames(result_f3) = c("simpID", "f3Num")

    # join the three tables together
    result_f1_f2_f3 = full_join(full_join(result_f1, result_f2, by='simpID'), result_f3, by='simpID')

    # Substitue NA to 0
    result_f1_f2_f3[is.na(result_f1_f2_f3)] = 0

    # Order by f1Num
    result_f1_f2_f3 = arrange(result_f1_f2_f3, desc(f1Num))

    # Add gene IDs
    idx1 = match(result_f1_f2_f3$simpID, mORF_table$simpID)

    result_f1_f2_f3 = result_f1_f2_f3 %>%
                    mutate(ens_gene = mORF_table$ens_gene[idx1], ext_gene = mORF_table$ext_gene[idx1], merged_orfID=mORF_table$merged_orfID[idx1])

    #Re-order column, easier to see f1Num
    result_f1_f2_f3 = result_f1_f2_f3[,c(2,3,4,5,6,1,7)]

    t=second_to_humanReadableTime(round((proc.time() - ptm)[3],2)) 
    cat(paste0(t[1],"h-",t[2],"m-",round(t[3]),"s\n"))
    
    # ---------------------------
    #output tsv
    fwrite(result_f1_f2_f3, paste0(opt$out,"_",id_vec[i],"_OrfCountByFrame.tsv"), sep="\t", quote = FALSE, row.names= FALSE)
    fwrite(gene_count_tbl, paste0(opt$out,"_",id_vec[i],"_GeneCount.tsv"), sep="\t", quote = FALSE, row.names= FALSE)
    
    # Number of in-frame reads 
    message(paste0("in-frame reads: ", nrow(result_inframe), " / ","total reads: ", nrow(result),
                   " (", round(nrow(result_inframe)/nrow(result),3)*100, "%)"))

}


Mapping reads to ORFs ... 2 input sam files
input 1 Reading reads ... 0h-42m-37s
input 1 Counting read numbers in gene ... 0h-42m-38s
input 1 Counting read numbers in ORF ... 0h-42m-46s


in-frame reads: 2435363 / total reads: 6273227 (38.8%)


input 2 Reading reads ... 0h-42m-47s
input 2 Counting read numbers in gene ... 0h-42m-48s
input 2 Counting read numbers in ORF ... 0h-42m-56s


in-frame reads: 2435363 / total reads: 6273227 (38.8%)


In [154]:
result_f1_f2_f3$merged_orfID[result_f1_f2_f3$ext_gene=="DDX3X"][1:5]

[1] "41334253:41334297:0|41337408:41337465:0|41339039:41339083:2|41341484:41341616:2|41342495:41342653:1|41342737:41342836:1|41343216:41343351:0|41343737:41343822:2|41344030:41344128:0|41344239:41344399:0|41345180:41345324:1|41345404:41345548:0|41346229:41346410:2|41346505:41346622:0|41346859:41347012:2|41347312:41347451:1|41347640:41347716:2|chrX;+"
[2] "41334253:41334297:0|41337408:41337465:0|41339039:41339083:2|41341484:41341616:2|41342495:41342653:1|41342737:41342836:1|41343216:41343351:0|41343737:41343822:2|41344030:41344128:0|41344239:41344399:0|41345180:41345324:1|41345404:41345548:0|41346229:41346410:2|41346505:41346622:0|41346859:41347012:2|41347315:41347451:1|41347640:41347716:2|chrX;+"
[3] "41334253:41334297:0|41337408:41337465:0|41339039:41339083:2|41341484:41341616:2|41342495:41342653:1|41342737:41342836:1|41343216:41343351:0|41343737:41343822:2|41344030:41344128:0|41344239:41344564:0|41345180:41345324:1|41345404:41345548:0|41346229:41346410:2|41346505:41346622:0|41346859:41347012:2|41347312:41347451:1|41347640:41347716:2|chrX;+"
[4] "41334253:41334297:0|41337408:41337465:0|41339036:41339083:2|41341484:41341616:2|41342495:41342653:1|41342737:41342836:1|41343216:41343351:0|41343737:41343822:2|41344030:41344128:0|41344239:41344399:0|41345180:41345324:1|41345404:41345548:0|41346229:41346410:2|41346505:41346622:0|41346859:41347012:2|41347315:41347454:1|41347640:41347716:2|chrX;+"
[5] "41334253:41334297:0|41337408:41337465:0|41339036:41339083:2|41341484:41341616:2|41342495:41342653:1|41342737:41342836:1|41343216:41343351:0|41343737:41343822:2|41344030:41344128:0|41344239:41344399:0|41345180:41345324:1|41345404:41345548:0|41346229:41346410:2|41346505:41346622:0|41346859:41347012:2|41347312:41347451:1|41347640:41347716:2|chrX;+"

In [148]:
result_f1_f2_f3[1:100,]

f1Num,f2Num,f3Num,ens_gene,ext_gene,simpID,merged_orfID
17894,61846,20449,XLOC_046810,novo_ncRNA10187,novo_ncRNA10187.13,8258257:8258313:0|chr21;+
5872,41,21,XLOC_003786,novo_ncRNA1744,novo_ncRNA1744.1,248874242:248874259:0|chr1;+
5872,41,21,XLOC_003786,novo_ncRNA1744,novo_ncRNA1744.2,248874254:248874259:0|chr1;+
5774,12,6,ENSG00000228549.3,BX284668.2,BX284668.2.1,16872580:16872594:0|chr1;+
5774,12,6,ENSG00000228549.3,BX284668.2,BX284668.2.2,16872589:16872594:0|chr1;+
3941,1979,2399,ENSG00000026025.16,VIM,VIM.4,17229423:17229985:0|17230650:17230710:1|17233587:17233682:0|17233770:17233931:0|17234693:17234818:0|17235169:17235389:0|17235846:17235889:1|17236294:17236379:2|17237230:17237268:0|chr10;+
3808,1905,2269,ENSG00000026025.16,VIM,VIM.19,17229423:17229985:0|17230650:17230710:1|17233587:17233682:0|17233770:17233931:0|17234693:17234818:0|17235169:17235389:0|17235846:17235909:1|chr10;+
3711,6522,6088,XLOC_046808,FP671120.4,FP671120.4.17,8209805:8210035:0|chr21;+
3699,6515,6082,XLOC_046808,FP671120.4,FP671120.4.18,8209811:8210035:0|chr21;+
3660,6470,6071,XLOC_046822,FP236383.3,FP236383.3.15,8437050:8437280:0|chr21;+


In [103]:
#output general info
gtf_CDS=data.frame(V1=CDS_blocks$chr,
                   V2="ArthurRiboPipeline",
                   V3="CDS",
                   V4=CDS_blocks$str,
                   V5=CDS_blocks$end,
                   V6=".",
                   V7=CDS_blocks$strand_col,
                   V8=".",
                   V9=paste0("gene_id \"",CDS_blocks$simp_id,"\"; transcript_id \"",CDS_blocks$simp_id,"\";gene_type \"",CDS_blocks$genetype,"\";"),
                   stringsAsFactors=F
                   )
gtf_CDS=gtf_CDS[order(gtf_CDS$V1,gtf_CDS$V4,decreasing = F),]

fwrite(gtf_CDS,file = paste0(opt$out,".gtf"),quote = F,sep="\t",row.names = F,col.names = F)

fwrite(RNA_gtf, paste0(opt$out,"_TranscriptInfoSummary.tsv"), sep="\t", quote = FALSE, row.names= FALSE)

In [99]:
table(CDS_blocks$genetype)

< table of extent 0 >

In [104]:
CDS_blocks$genetype[1:5]

[1] "protein_coding" "protein_coding" "protein_coding" "protein_coding"
[5] "protein_coding"